In [1]:
import string
from pickle import load

In [2]:
def load_file(filename):
    file = open(filename,'r' , encoding='ISO-8859-1')
    text = file.read()
    file.close()
    return text

def collection_data(text):
    mapping = dict()
    for line in text.split('\n'):            
        token = line.split()
        if len(line) < 2:
            continue
        image_id,image_desc = token[0],token[1:]
        image_id = token[0].split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    return mapping

In [3]:
def clean_data(desc):
    table = str.maketrans('','',string.punctuation)
    for key,desc_list in desc.items():
        for i in range(len(desc_list)):
            text=desc_list[i]
            text=text.split()
            text=[word.lower() for word in text]
            text=[word.translate(table) for word in text]
            text=[word for word in text if len(word) > 2]
            text=[word for word in text if word.isalpha()]
            desc_list[i] = ' '.join(text)
    return desc

In [4]:
def save_text(desc,filename):
    lines = list()
    file = open(filename,'w')
    for key,desc_list in desc.items():
        for i in desc_list:
            string = key + ' ' + i
            lines.append(string)
            file.write(string + '\n')
    file.close()

In [5]:
def to_voc(desc):
    all_desc = set()
    for key in desc.keys():
        [all_desc.update(d.split()) for d in desc[key]]
    return all_desc

In [6]:
filename = 'Flicker8k/captions.txt'
doc = load_file(filename)
description = collection_data(doc)
clean_desc = clean_data(description)
voc = to_voc(description)

In [7]:
len(description)

8198

In [8]:
file = 'Flicker8k/clean_data.txt'
save_text(clean_desc,file)

In [9]:
def load_set(filename):
    doc = load_file(filename)
    dataset = list()
    for line in doc.split('\n'):
        if len(line) < 1 :
            continue
        identifer = line.split('.')[0]
        dataset.append(identifer)
    return set(dataset)

In [10]:
def load_desc(filename,dataset):
    doc = load_file(filename)
    desc = dict()
    for line in doc.split('\n'):
        token=line.split()
        image_id , image_desc = token[0],token[1:]
        if image_id in dataset:
            if image_id not in desc:
                desc[image_id] = list()
            desc_end = 'startseq ' + ' '.join(image_desc) + ' endseq'
            desc[image_id].append(desc_end)
    return desc

In [11]:
def load_features(filename,dataset):
    all_features = load(open(filename,'rb'))
    features = {k:all_features[k] for k in dataset}
    return features

In [12]:
trainfile = 'Flicker8k/id.txt'
train = load_set(trainfile)
print("Dataset : {}".format(len(train)))

Dataset : 8198


In [13]:
train_desc = load_desc('Flicker8k/clean_data.txt',train)
print("Descriptions : {}".format(len(train_desc)))

Descriptions : 8198


In [14]:
photo_features = load_features('Flicker8k/Features8K.pkl',train)
print("Features = {}".format(len(photo_features)))

Features = 8198


In [15]:
keys = list(clean_desc.keys())

In [16]:
split = int(len(keys) * 0.90)
train = keys[:split]
validate = keys[split:]

In [17]:
from numpy import array
import numpy as np
import keras
from tqdm import tqdm
import tensorflow
from pickle import load 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model , to_categorical
from tensorflow.keras.models import Model , load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense , Input , LSTM , Embedding , Dropout ,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import add 
from tensorflow.keras.preprocessing.image import load_img , img_to_array 
from tensorflow.keras.applications.mobilenet import preprocess_input
import pickle

In [18]:
def to_lines(desc):
    all_desc = list()
    counter = 0
    for key in desc.keys():
        [all_desc.append(d) for d in desc[key]]
    return all_desc

In [19]:
all_desc = to_lines(train_desc)
print(len(all_desc))

40990


In [20]:
print(all_desc[0])

startseq lots cars parked the street endseq


In [21]:
def create_tokenizer(desc):
    lines = to_lines(desc)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [22]:
tokenizer = create_tokenizer(train_desc)

In [23]:
pickle.dump(tokenizer, open('Flicker8k/tokenizers.p', 'wb'))

In [24]:
max_length = max(len(caption.split()) for caption in all_desc)
max_length

28

In [25]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8828


In [26]:
def create_seq(token, max_length, desc_list, photo):
    x1, x2, y = list(), list(), list()
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq , out_seq = seq[:i] , seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            x1.append(photo)
            x2.append(in_seq)
            y.append(out_seq)
    return array(x1), array(x2), array(y)

In [33]:
def define_model(vocab_size, max_length):
    inputs1 = Input(shape=(1280,))
    fe1 = Dropout(0.1)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    fe3 = BatchNormalization()(fe2)
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.4)(se1)
    se3 = LSTM(256)(se2)
    se4 = BatchNormalization()(se3)
    decoder1 = add([fe3, se4])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    model = keras.models.Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model 

In [34]:
model = define_model(vocab_size,max_length)

In [35]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 28)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1280)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 28, 256)      2259968     ['input_4[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 1280)         0           ['input_3[0][0]']                
                                                                                            

In [36]:
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in data_keys:
            n += 1
            captions = mapping[key]
            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], 
                    num_classes=vocab_size)[0]
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield [X1, X2], y
                X1, X2, y = list(), list(), list()
                n = 0

In [37]:
epochs = 40
batch_size = 64
steps = len(train) // batch_size
for i in range(epochs):
    print("Epoch {} / {}".format(i+1 , epochs))
    generator_train = data_generator(train, train_desc, photo_features, tokenizer, max_length, vocab_size, batch_size)
    model.fit(generator_train, epochs=1, steps_per_epoch=steps, verbose=1)
model.save('Flicker8k/models/new model.h5')

Epoch 1 / 40
115/115 [==============================] - 433s 4s/step - loss: 5.4935 - accuracy: 0.1721
Epoch 2 / 40
115/115 [==============================] - 420s 4s/step - loss: 4.2000 - accuracy: 0.2587
Epoch 3 / 40
115/115 [==============================] - 423s 4s/step - loss: 3.7304 - accuracy: 0.2904
Epoch 4 / 40
115/115 [==============================] - 423s 4s/step - loss: 3.4030 - accuracy: 0.3123
Epoch 5 / 40
115/115 [==============================] - 423s 4s/step - loss: 3.1335 - accuracy: 0.3329
Epoch 6 / 40
115/115 [==============================] - 420s 4s/step - loss: 2.9001 - accuracy: 0.3579
Epoch 7 / 40
115/115 [==============================] - 423s 4s/step - loss: 2.7162 - accuracy: 0.3847
Epoch 8 / 40
115/115 [==============================] - 419s 4s/step - loss: 2.5770 - accuracy: 0.4055
Epoch 9 / 40
115/115 [==============================] - 425s 4s/step - loss: 2.4616 - accuracy: 0.4228
Epoch 10 / 40
115/115 [==============================] - 420s 4s/step - l

In [42]:
model = load_model('Flicker8k/Describe Scene.h5')

In [38]:
def extract_feature(filename,model):
    image = Image.open(filename)
    image = image.resize((224,224))
    image = img_to_array(image)
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image,verbose=1)
    return feature

In [39]:
def generate_caption(model, tokenizer, photo_features, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo_features, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    caption = in_text.replace('startseq', '').replace('endseq', '')
    return caption

In [40]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
actual, predicted = list(), list()
for key in tqdm(validate):
    captions = clean_desc[key]
    y_pred = generate_caption(model, tokenizer, photo_features[key], max_length)
    actual_captions = [caption.split() for caption in captions]
    y_pred = y_pred.split()
    actual.append(actual_captions)
    predicted.append(y_pred)
    print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))

  0%|                                                                                  | 1/820 [00:01<19:17,  1.41s/it]

BLEU-1: 1.000000
BLEU-2: 0.707107


  0%|▏                                                                                 | 2/820 [00:01<10:18,  1.32it/s]

BLEU-1: 0.818731
BLEU-2: 0.578930


  0%|▎                                                                                 | 3/820 [00:02<08:21,  1.63it/s]

...

BLEU-1: 0.751147
BLEU-2: 0.634653


 46%|█████████████████████████████████████▏                                          | 381/820 [03:03<04:20,  1.69it/s]

BLEU-1: 0.751811
BLEU-2: 0.635668


 47%|█████████████████████████████████████▎                                          | 382/820 [03:04<04:21,  1.67it/s]

BLEU-1: 0.750665
BLEU-2: 0.634457
